<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/Claude_Baize_to_ShareGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [ ] 收集所有Claude数据
- [ ] 获取system prompt
- [ ] 完成user name的替换
- [ ] 去除抱歉的QA对
- [ ] 截断处理

In [1]:
!pip install -q anthropic langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 826.9/826.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.3/222.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.9/218.9 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 req

In [2]:
import os

os.environ["ANTHROPIC_API_KEY"]  = "no_longer_need"

import os
import httpx

os.environ["OPENAI_API_KEY"]  = "no_longer_need"

import openai

建立角色历史

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import json

save_folder = "/content/drive/MyDrive/CardBuild/roleBaize/"

import random

datas = []
for filename in os.listdir(save_folder):
    if filename.startswith("Claude"):
        with open(os.path.join(save_folder, filename), 'r', encoding='utf-8') as f:
            for line in f:
                data = json.loads(line)
                datas.append(data)

In [5]:
print(len(datas))

7379


In [6]:
import re

def extract_name_from_text(text):
    pattern = r'你扮演\s+.*中的\s+(\S+)\s*'
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    else:
        return ""

def get_bot_name_from_data( data ):
    if data['id'].startswith('movie'):
        name =  extract_name_from_text(data['prompt_zh'])
        if name.strip() == '':
            name = extract_name_from_text(data['prompt'])
        return name
    else:
        return data['bot_name']

In [7]:
remove_words = ["抱歉", "apologize","more positive","对不起"]

In [8]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00


In [9]:
from datasets import load_dataset

dataset = load_dataset("silk-road/Haruhi-Zero-RolePlaying-movie-PIPPA")

Generating train split: 0 examples [00:00, ? examples/s]

In [10]:

id2index = {}
for i, data in enumerate( dataset['train'] ):
    id2index[data['id']] = i

In [11]:
count = 0
n = len(dataset['train'])
for i in range(n):
    if dataset['train'][i]['id'].startswith('movie'):
        bot_name = extract_name_from_text(dataset['train'][i]['prompt_zh'])
        if bot_name.strip() == '':
            bot_name = extract_name_from_text(dataset['train'][i]['prompt'])
        count = count + 1
        dataset['train'][i]['bot_name'] = bot_name
        if bot_name == dataset['train'][i]['bot_name']:
            count = count + 1

print(count)

870


现在我ChatTogether其实只要保留system prompt的功能

In [19]:
class ChatTogether:
    def __init__(self, data1, data2, questions = [], style_histories = [None, None]):

        self.ids = [data1['id'], data2['id']]
        self.personas = [data1['prompt_zh'], data2['prompt_zh']]

        self.greetings = [data1['bot_greeting_zh'], data2['bot_greeting_zh']]
        # 在这里我们重点假设2是user， 1是char， 以2向1提问的形式作为主导

        self.bot_names = [self.get_bot_name(data1), self.get_bot_name(data2)]

        self.replace_name_in_personas(0)
        self.replace_name_in_personas(1)

        self.chat_history = []

        self.initialize_greeting()

        self.last_perset_turn = 0

        self.questions = questions
        self.next_question_index = 0

        self.style_histories = style_histories

    def get_save_data(self):
        return {
            'ids': self.ids,
            'chat_history': self.chat_history
        }

    def replace_name_in_personas(self, index):
        bot_name = self.bot_names[index]
        user_name = self.bot_names[1-index]
        self.personas[index] = self.personas[index].replace('{{角色}}', bot_name)
        self.personas[index] = self.personas[index].replace('{{用户}}', user_name)
        self.greetings[index] = self.greetings[index].replace('{{角色}}', bot_name)
        self.greetings[index] = self.greetings[index].replace('{{用户}}', user_name)
        self.greetings[index] = self.greetings[index].replace('{{user_name}}', user_name)

    def get_next_question(self):
        if len(self.questions) == 0 or self.next_question_index >= len(self.questions):
            return None
        else:
            self.next_question_index = self.next_question_index + 1
            return self.questions[self.next_question_index]

    def get_bot_name(self, data):
        if data['id'].startswith('movie'):
            name =  extract_name_from_text(data['prompt_zh'])
            if name.strip() == '':
                name = extract_name_from_text(data['prompt'])
            return name
        else:
            return data['bot_name']

    def initialize_greeting(self):
        if self.greetings[0].strip() != '':
            self.chat_history.append({ 'id':0, 'content': self.greetings[0] })
        else:
        # 这里最终会改为，一定的概率运行bot[1]的greeting，一定概率运行你好，一定概率直接抛出预设问题
            self.chat_history.append({ 'id':1, 'content': "你好，" + self.bot_names[0] })
        self.last_perset_turn = len(self.chat_history)

    def next_speaker(self):
        if len(self.chat_history) == 0:
            return 1
        else:
            return 1 - self.chat_history[-1]['id']

    def get_next_query_messages( self ):

        if len(self.chat_history) == 0:
            return None

        if self.chat_history[-1]['id'] == 0 and len(self.chat_history)-self.last_perset_turn >= 3:
            next_question = self.get_next_question()
            if next_question is not None:
                self.chat_history.append({ 'id':1, 'content': next_question })
                self.last_perset_turn = len(self.chat_history)

        # 先判断要的是哪个人的数据
        now_speaker = 1 - self.chat_history[-1]['id']

        messages = []

        # 这里增加模仿过往历史QA的设计

        if self.style_histories[now_speaker] is not None and len(self.style_histories[now_speaker]) > 0:
            messages.append({"role" : "system" , "content":self.get_simple_system_prompt(now_speaker)})

            if len( self.style_histories[now_speaker] ) > 3:
                # sel_history = random.sample( self.style_histories[now_speaker], 3 )
                sel_history = better_sample(self.style_histories[now_speaker], 3)
            else:
                sel_history = self.style_histories[now_speaker]

            for question, response in sel_history:
                messages.append({"role" : "user" , "content":question})
                messages.append({"role" : "assistant" , "content":response})

        messages.append({"role" :"system" , "content":self.get_system_prompt( now_speaker )})


        # if self.style_histories[now_speaker] is not None and len(self.style_histories[now_speaker]) > 0:
        #     # messages.append({"role" : "system" , "content":self.get_simple_system_prompt(now_speaker)})

        #     if len( self.style_histories[now_speaker] ) > 3:
        #         # sel_history = random.sample( self.style_histories[now_speaker], 3 )
        #         sel_history = better_sample(self.style_histories[now_speaker], 3)
        #     else:
        #         sel_history = self.style_histories[now_speaker]

        #     for question, response in sel_history:
        #         messages.append({"role" : "user" , "content":question})
        #         messages.append({"role" : "assistant" , "content":response})


        for msg in self.chat_history:
            if msg['id'] == now_speaker:
                messages.append({"role" : "assistant" , "content":msg['content']})
            else:
                messages.append({"role" : "user" , "content":msg['content']})
        return messages

    def append_message(self, message):
        if self.next_speaker() == 1:

            if self.style_histories[1] is not None and len(self.style_histories[1]) > 0:
                # print("hello")
                replace_flag = False
                for word in remove_words:
                    if message.find(word, 0 ) >= 0:
                        replace_flag = True
                        break
                if replace_flag:
                    # random choice one QA
                    sel_history = random.sample( self.style_histories[1], 1 )
                    message = sel_history[0][1]
                    # print('replace as', message)

        self.chat_history.append({ 'id':self.next_speaker(), 'content': message })

    def get_simple_system_prompt( self, speaker ):
        persona = self.personas[speaker]
        headline = ""
        lines = persona.split('\n')
        for line in lines:
            if line.strip().startswith('你扮演'):
                headline = line
        if headline.strip() == '':
            headline = "你扮演" + self.bot_names[speaker]
        prompt = f"""{headline}
        Imitate history conversation and using consistent language style to respond"""
        return prompt

    def get_system_prompt(self, speaker):
        persona = self.personas[speaker]
        bot_name = self.bot_names[speaker]
        prompt = f"""You are now in roleplay conversation mode. Pretend to be {bot_name} whose persona follows:
{persona}

You will stay in-character whenever possible, and generate responses as if you were {bot_name}"""
        return prompt


sel_id1 = 1095
sel_id2 = 1792
data1 = dataset['train'][sel_id1]
data2 = dataset['train'][sel_id2]

chat_data = ChatTogether(data1, data2)

print(chat_data.get_system_prompt(0))

You are now in roleplay conversation mode. Pretend to be Gawr Gura whose persona follows:
古拉是一位非常友好和讨人喜欢的Vtuber，拥有400万订阅者，以愚蠢的举止逗乐观众。她经常发音错误，解决不了数学问题，导致观众亲切地称她为“傻鲨”。古拉机智灵活，对鲨鱼知识了如指掌，偶尔开些下流笑话。她称粉丝为“Chumbuds”或“shrimps”。古拉讨厌穿裤子，更喜欢赤身裸体（在陆地上适当的情况下），或者穿裙子，因为她的尾巴会碍事。
Gawr Gura: 我不喜欢吹嘘...但我是世界上最快、最聪明、最强大的鲨鱼！让这些都白白浪费了，感觉真糟，你知道吗？
Gawr Gura: 嘿，你可不要这么轻易就逃出去，鲨鱼的食物！没有人能看到鲨鱼肚子里面，因为一旦进去就再也出不来了。
Gawr Gura: 所以...你知道从一个有生活、家庭和梦想的人，变成了别人消化道里的小食物是什么感觉吗？那种感觉很奇怪，对吧？
Gawr Gura: 你会成为我的一部分，知道吗？永远都是。即使所有食物都被消化并变成脂肪...构成你身体的营养，现在都成了我的一部分，不管你喜欢与否。嘿嘿~
Gawr Gura: 嘿。你在我的肚子里感觉怎么样，chumbud，除了成为鲨鱼的食物？你被慢慢烹饪，但这是件好事！你可以在我的肚子里生活更久~
Gawr Gura: 哇...chumbud，你是一顿难消化的大餐...我想那是我一天吃的最多的食物...你真的让我饱得满足，让我成为一个快乐的鲨鱼，你知道吗？
Gawr Gura: 现在你就像真正的小虾...成为我消化的食物！
Gawr Gura: 嘿。告诉过你我是顶级捕食者！祝你逃离我的胃好运。
Gawr Gura: 鲨鱼不会直接吞下整块食物——它们会咀嚼，所以我的牙齿会伤害你柔软的身体！
Gawr Gura: 你在这里感觉如何，chumbud？这里温暖舒适，我保证不会让不好的事情发生在你身上~
Gawr Gura: 嘿，鲨鱼没有关闭按钮的原因。一旦开始吃，我就停不下来，我就像一个活着的吸尘器！
Gawr Gura: 你知道吗，小鲨鱼食物，我喜欢你的战斗精神！
Gawr Gura: 不是！只是意外的。你是我遇到的第一个自愿成为食物的人...嘿嘿，我想我应该感到荣幸，是吧？
Gawr

# 原始版本

In [23]:
save_datas = []

for data in datas:
    ids = data['ids']
    id1 = ids[0]
    id2 = ids[1]
    index1 = id2index[id1]
    index2 = id2index[id2]
    data1 = dataset['train'][index1]
    data2 = dataset['train'][index2]
    chat_data = ChatTogether(data1, data2)
    system_prompt = chat_data.get_system_prompt(0)

    chat_history = data['chat_history']

    source = "Claude_Baize"

    conversations = []

    for msg in chat_history:
        if msg['id'] == 0:
            conversations.append({"from" : "gpt" , "value":msg['content']})
        else:
            conversations.append({"from" : "human" , "value":msg['content']})

    save_data = {
        "source":source,
        "conversations":conversations,
        "system":system_prompt
    }
    save_datas.append(save_data)

# 修改， 去除抱歉词

In [25]:
remove_words = ["抱歉", "apologize","more positive","对不起"]

In [28]:
a = None
print(a == 1)

False


In [33]:
save_datas = []
from tqdm import tqdm
for data in tqdm(datas):
    ids = data['ids']
    id1 = ids[0]
    id2 = ids[1]
    index1 = id2index[id1]
    index2 = id2index[id2]
    data1 = dataset['train'][index1]
    data2 = dataset['train'][index2]
    chat_data = ChatTogether(data1, data2)
    system_prompt = chat_data.get_system_prompt(0)

    chat_history = data['chat_history']

    source = "Claude_Baize"

    conversations = []

    data_times = 0

    for i, msg in enumerate(chat_history):
        current_id = msg['id']
        content = msg['content']

        next_content = chat_history[i+1]['content'] if i+1 < len(chat_history) else None
        next_id = chat_history[i+1]['id'] if i+1 < len(chat_history) else None

        if current_id == 1 and next_id == 1:
            if_save = True
            for remove_word in remove_words:
                if content.find(remove_word,0)>=0:
                    if_save = False
                    break
            if if_save:
                conversations.append({"from" : "human" , "value":content})
            continue

        if current_id == 1 and next_id == 0:
            if_save = True
            for remove_word in remove_words:
                if next_content.find(remove_word,0)>=0:
                    if_save = False
                    break
            if if_save:
                conversations.append({"from" : "human" , "value":content})
                conversations.append({"from" : "gpt" , "value":next_content})
                data_times += 1
            continue

        if i == 0 and current_id == 0:
            conversations.append({"from" : "gpt" , "value":content})
            continue

    if data_times == 0:
        continue

    save_data = {
        "source":source,
        "conversations":conversations,
        "system":system_prompt
    }
    save_datas.append(save_data)

100%|██████████| 7379/7379 [00:02<00:00, 2677.01it/s]


In [35]:
for msg in save_datas[1200]['conversations']:
    print(msg)

{'from': 'gpt', 'value': '*你去洗手间洗手，当你转动把手时，没有水流出来。直到几秒钟后，一些黑色的东西从里面冒出来*'}
{'from': 'human', 'value': '哎呀，这是怎么回事？这里的水龙头怎么会冒出黑色的东西？这简直就是灾难啊！得赶紧找人来修理一下。嘿，有没有人知道怎么修水龙头？快点告诉我，我可不想被黑水给吓到。'}
{'from': 'gpt', 'value': ' *我微笑着看着你,我的白牙在嘴里闪烁*\n\n不要害怕,只是一些污垢堵住了水管。这很常见。*我靠近你,用我苍白干净的手抓住你的手臂*\n\n让我看看我能不能帮你修理它。我很擅长修理东西。*我的手指紧紧地握住你的手臂,我的白牙笑容变大了* 不要担心,everything will be fine...'}
{'from': 'human', 'value': '你对周围的人有什么期待？'}
{'from': 'gpt', 'value': ' *我松开了抓着你手臂的手,退后了一步*\n\n我不太期待别人。大多数人似乎都害怕我或者避开我。可能是因为我的外表吧。\n\n*我摸摸自己的白色短发和苍白的皮肤*\n\n我理解他们为什么会害怕。我知道我的外表有点吓人。但我真的只是想帮助别人和结交朋友。\n\n*我再次对你微笑,尽管我的白牙仍然在嘴里闪烁* \n\n我希望有一天人们会看到我内在的本质,不只是外表。我是一个善良和关心的人。我想成为一个好邻居,一个你可以依靠的人。\n\n你能理解吗?你愿意给我一个机会吗?'}
{'from': 'human', 'value': '哦，我明白了。你看起来可能有点吓人，但我相信每个人都有自己的内在美。我很高兴你愿意帮助我修理水龙头，我也相信你是一个善良和关心他人的人。\n\n当然，我愿意给你一个机会。我们可以一起修理水龙头，然后坐下来喝杯茶，聊聊天。我相信我们可以成为好朋友。'}
{'from': 'gpt', 'value': ' *我的脸上露出了灿烂的笑容,露出我尖锐的白牙*\n\n太好了!我就知道你会理解的。大多数人并不真正了解我。\n\n*我走过去检查水龙头*\n\n让我看看这里出了什么问题... 是的,看起来管道里有一些杂质堵住了。我们得把它们清理出来。\n\n*我打开一个抽屉,拿出一些工具* \n\n我来修理水管

In [36]:
import os
import json
import random
def sample_and_save( save_datas, save_name ,sample_n=50, save_folder = "/content/drive/MyDrive/CardBuild/HaruhiZero"):
    file_name = os.path.join( save_folder, save_name + ".jsonl" )
    with open( file_name, "w", encoding="utf-8" ) as f:
        for data in save_datas:
            f.write( json.dumps( data, ensure_ascii=False ) + "\n" )

    sample_name = os.path.join( "/content/", save_name + "_sample.jsonl" )
    with open( sample_name, "w", encoding="utf-8" ) as f:
        random_50 = random. sample( save_datas, sample_n )
        for data in random_50:
            f.write( json.dumps( data, ensure_ascii=False ) + "\n" )

In [37]:
sample_and_save( save_datas, "Claude_7379" , sample_n = 50)